In [ ]:
import pandas as pd
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
import os

from fbprophet.plot import plot_plotly
import plotly.offline as py
from plotly.offline import iplot, plot, download_plotlyjs, init_notebook_mode

In [ ]:
filepath = "C:/Users/user/Downloads/sales-wawa.csv"
output = "C:/Users/user/Downloads/"
df = pd.read_csv(filepath)

df_sales = df.dropna()
df_sales = df_sales[['store_number','date','sales_index']]
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [ ]:
def forecast(stores,m,df):
    store = df.loc[df['store_number'] ==  stores][['date','sales_index']]
    store.rename(columns={'date':'ds', 'sales_index':'y'}, inplace=True)
    m.fit(store)
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    m.plot(forecast)
    m.plot_components(forecast)
    return forecast

def validation(m):
    df_cv = cross_validation(m, horizon='90 days')
    df_p = performance_metrics(df_cv)
    return df_p

In [ ]:
store_num = df_sales.store_number.unique().tolist()
for stores in store_num:
    m = Prophet()
    df_f = forecast(stores,m, df_sales)
    df_p = validation(m)
    
    #Interactive Plotly Graph
    py.init_notebook_mode()
    fig = plot_plotly(m, df_f)
    py.iplot(fig)
    
    #Writing Output to Excel Files
    if not os.path.exists(output+"\\"+str(stores)):
        os.makedirs(output+"\\"+str(stores))
        df_p.to_excel(output+"\\"+str(stores)+"\\"+str(stores)+"_p.xlsx",sheet_name='Prediction')
        df_f.to_excel(output+"\\"+str(stores)+"\\"+str(stores)+"_f.xlsx",sheet_name='Forecast')